# Linear models last time step prediction

In [1]:
import matplotlib.pyplot as plt
import json
import os
import datetime
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import metrics
data_dir = './vaccine_data/hhs_data/'

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 999;

<IPython.core.display.Javascript object>

In [3]:
train_years = ['2013','2014']
dev_years = ['2015']
test_years = ['2016','2017']

train_download_dir = ('train',train_years)
dev_download_dir = ('dev',dev_years)
test_download_dir = ('test',test_years)
data_split_ls = [train_download_dir,dev_download_dir,test_download_dir]

In [6]:
split = 'train'
train_years = ['2013','2014']
X = []
for year in train_years:
    data_df = pd.read_json(open(os.path.join(data_dir,split,year,'national.json')))
    X.append(data_df['percentage'].values)
X = np.asarray(X)
week_means = X.mean(axis = 0)

threshold_start_pred  = 1 


In [8]:
split,years = train_download_dir
threshold_start_pred  = 4 
Y = []
X = []
for year in years:
    data = pd.read_json(open(os.path.join(data_dir,split,year,'national.json')))
    percent_vaccinated_week = (data['percentage']).values
    percent_vaccinated_week = percent_vaccinated_week -  week_means

    initial_week_vals = [percent_vaccinated_week[i] for i in range(0,threshold_start_pred)]
    
    for i in range(threshold_start_pred,len(percent_vaccinated_week)):
        X.append([i]+initial_week_vals)
        Y.append(percent_vaccinated_week[i])


X = np.asarray(X)
Y = np.asarray(Y)
model = linear_model.LinearRegression()
model.fit(X,Y)
y_pred = model.predict(X)
error = metrics.mean_squared_error([y_pred[-1]],[Y[-1]])
print("Error in last train prediction {} %".format(error*100 ))

Error in last train prediction 0.000203455752710052 %


## Dev Prediction

In [6]:
split = 'dev'
year = '2015'
data = pd.read_json(open(os.path.join(data_dir,split,year,'national.json')))
percent_vaccinated_week = (data['percentage']).values
percent_vaccinated_week = percent_vaccinated_week -  week_means

initial_week_vals = [percent_vaccinated_week[i] for i in range(0,threshold_start_pred)]
Y = []
X = []
for i in range(threshold_start_pred,len(percent_vaccinated_week)):
    X.append([i]+initial_week_vals)
    Y.append(percent_vaccinated_week[i])

X = np.asarray(X)
Y = np.asarray(Y)

y_pred = model.predict(X)
error = metrics.mean_squared_error([y_pred[-1]],[Y[-1]])
print("Error in last dev prediction {} %".format(error*100 ))

Error in last dev prediction 0.03315038673220914 %


## Test Prediction

In [11]:
split = 'test'
years = ['2016','2017']
for year in years:
    data = pd.read_json(open(os.path.join(data_dir,split,year,'national.json')))
    percent_vaccinated_week = (data['percentage']).values
    percent_vaccinated_week = percent_vaccinated_week -  week_means[0:len(percent_vaccinated_week)]

    initial_week_vals = [percent_vaccinated_week[i] for i in range(0,threshold_start_pred)]
    Y = []
    X = []
    for i in range(threshold_start_pred,len(percent_vaccinated_week)):
        X.append([i]+initial_week_vals)
        Y.append(percent_vaccinated_week[i])

    X = np.asarray(X)
    Y = np.asarray(Y)
    y_pred = model.predict(X)
    error = metrics.mean_squared_error([y_pred[-1]],[Y[-1]])
    print("Error in last test prediction {} %".format(error*100 ))

Error in last test prediction 0.028045016905053986 %
Error in last test prediction 0.1545675874196898 %


In [12]:
print("Model Coeff {},".format(model.coef_))

Model Coeff [-1.05124778e-19 -3.14996031e-02 -6.43303807e-02 -7.30701974e-02
 -3.34019620e-02],
